In [1]:
!pip install torch
!pip install git+https://github.com/openai/shap-e.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/shap-e.git to /tmp/pip-req-build-uz0d_ihe
  Running command git clone --filter=blob:none --quiet https://github.com/openai/shap-e.git /tmp/pip-req-build-uz0d_ihe
  Resolved https://github.com/openai/shap-e.git to commit 8625e7c15526d8510a2292f92165979268d0e945
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-jffmho63/clip_332e25965a944a788cb49008af9ef60c
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-jffmho63/clip_332e25965a944a788cb49008af9ef60c
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 6

In [2]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|███████████████████████████████████████| 890M/890M [00:11<00:00, 81.3MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [5]:
batch_size = 4
guidance_scale = 15.0
prompt = "a shark"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [6]:
render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIMAAAAAAAAzAAAzMzMzMwAzZjMzZgBmZjNmZmZmZgAzmTMzmQBmmT…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIQAAAAAAAAzAAAAMwAzMzMzMwAzZjMzZgBmZjNmZmZmZmaZZjNmmW…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIMAAAAAAAAzMzMzMzNmZmZmZjOZZmaZZjNmmWZmmTOZmWaZmZmZmZ…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIQAAAAAAAAzAAAAMwAzMzMzMwAzZjMzZgBmZjNmZmZmZjNmmWZmmT…

In [7]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        decode_latent_mesh(xm, latent).tri_mesh().write_ply(f)

/usr/local/lib/python3.10/dist-packages/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/usr/local/lib/python3.10/dist-packages/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(
